In [1]:
import glob
import cv2
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)

In [2]:
image_path_list = glob.glob("competition_data/train/images/*")
X_train=[]
y_train=[]
for image_path in image_path_list[:100]:
    image = cv2.resize(cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)/255, (256, 256))
    X_train.append(image)
    mask = cv2.resize(cv2.imread(image_path.replace("images","masks"), cv2.IMREAD_GRAYSCALE)/255, (256, 256))
    y_train.append(mask)
X_train=np.array(X_train)
y_train=np.array(y_train)
X_train = np.expand_dims(X_train, axis=-1)
y_train = np.expand_dims(y_train, axis=-1)

In [3]:

from tensorflow.keras.utils import Sequence

class SaltDataset(Sequence):
    def __init__(self, image_dir, mask_dir, image_ids, batch_size=16, img_size=(128, 128), shuffle=True):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_ids = image_ids
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return len(self.image_ids) // self.batch_size

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.image_ids)

    def __getitem__(self, idx):
        batch_ids = self.image_ids[idx * self.batch_size:(idx + 1) * self.batch_size]
        X = np.zeros((self.batch_size, *self.img_size, 1), dtype=np.float32)
        Y = np.zeros((self.batch_size, *self.img_size, 1), dtype=np.float32)

        for i, img_id in enumerate(batch_ids):
            img = cv2.imread(os.path.join(self.image_dir, img_id + ".png"), cv2.IMREAD_GRAYSCALE)
            mask = cv2.imread(os.path.join(self.mask_dir, img_id + ".png"), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, self.img_size) / 255.0
            mask = cv2.resize(mask, self.img_size) / 255.0
            X[i, ..., 0] = img
            Y[i, ..., 0] = mask

        return X, Y


In [4]:
print(os.getcwd())


c:\Users\LOUIS J MAHOBE\Desktop\unet master


In [6]:


import os

def get_image_ids(image_dir):
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]
    image_ids = [os.path.splitext(f)[0] for f in image_files]
    return image_ids

In [7]:
train_ids = get_image_ids("C:/Users/LOUIS J MAHOBE/unet master")


In [8]:
import matplotlib.pyplot as plt

def visualize_sample(model, dataset, index=0):
    X, Y = dataset[index]
    preds = model.predict(X)

    fig, axs = plt.subplots(3, 5, figsize=(15, 9))
    for i in range(5):
        axs[0, i].imshow(X[i].squeeze(), cmap='gray')
        axs[0, i].set_title("Input")
        axs[1, i].imshow(Y[i].squeeze(), cmap='gray')
        axs[1, i].set_title("Ground Truth")
        axs[2, i].imshow(preds[i].squeeze(), cmap='gray')
        axs[2, i].set_title("Prediction")
        for j in range(3):
            axs[j, i].axis('off')
    plt.tight_layout()
    plt.show()


In [ ]:
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

def get_feature_maps(model, layer_name, input_image):
    

    intermediate_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    feature_maps = intermediate_model.predict(np.expand_dims(input_image, axis=0))
    return feature_maps[0]

def plot_feature_maps(feature_maps, num_maps=16):
    
    fig, axs = plt.subplots(1, num_maps, figsize=(20, 5))
    for i in range(num_maps):
        axs[i].imshow(feature_maps[..., i], cmap='viridis')
        axs[i].axis('off')
    plt.tight_layout()
    plt.show()


In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout
from keras.models import Model

inputs = Input(shape=(256, 256, 1))

# Encoder
conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

# Bottleneck
conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
drop5 = Dropout(0.5)(conv5)

# Decoder
up6 = UpSampling2D(size=(2, 2))(drop5)
up6 = Conv2D(512, 2, activation='relu', padding='same')(up6)
merge6 = concatenate([drop4, up6], axis=3)
conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

up7 = UpSampling2D(size=(2, 2))(conv6)
up7 = Conv2D(256, 2, activation='relu', padding='same')(up7)
merge7 = concatenate([conv3, up7], axis=3)
conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

up8 = UpSampling2D(size=(2, 2))(conv7)
up8 = Conv2D(128, 2, activation='relu', padding='same')(up8)
merge8 = concatenate([conv2, up8], axis=3)
conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

up9 = UpSampling2D(size=(2, 2))(conv8)
up9 = Conv2D(64, 2, activation='relu', padding='same')(up9)
merge9 = concatenate([conv1, up9], axis=3)
conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)
conv9 = Conv2D(2, 3, activation='relu', padding='same')(conv9)
conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)


model = Model(inputs=inputs, outputs=conv10)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [43]:
model = unet()


In [44]:
print([layer.name for layer in model.layers])


['input_6', 'conv2d_71', 'conv2d_72', 'max_pooling2d_17', 'conv2d_73', 'conv2d_74', 'max_pooling2d_18', 'conv2d_75', 'conv2d_76', 'max_pooling2d_19', 'conv2d_77', 'conv2d_78', 'dropout_6', 'max_pooling2d_20', 'conv2d_79', 'conv2d_80', 'dropout_7', 'up_sampling2d_9', 'conv2d_81', 'concatenate_9', 'conv2d_82', 'conv2d_83', 'up_sampling2d_10', 'conv2d_84', 'concatenate_10', 'conv2d_85', 'conv2d_86', 'up_sampling2d_11', 'conv2d_87', 'concatenate_11', 'conv2d_88', 'conv2d_89', 'up_sampling2d_12', 'conv2d_90', 'concatenate_12', 'conv2d_91', 'conv2d_92', 'conv2d_93', 'conv2d_94']


In [45]:
import glob

image_list = glob.glob("C:/Users/LOUIS J MAHOBE/unet master/unet/images/*.png")
print(image_list[:5])  


[]


In [79]:
image_path_list = glob.glob("competition_data/train/images/*")

In [82]:
import cv2
import numpy as np


image_path_list = glob.glob("competition_data/train/images/*")
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

if img is None:
    print("Image not found or failed to load:", image_path)
else:
    img = cv2.resize(img, (256, 256)) / 255.0
    img = np.expand_dims(img, axis=-1)  # shape: (256, 256, 1)


In [83]:
[layer.name for layer in model.layers]


['input_9',
 'conv2d_113',
 'conv2d_114',
 'max_pooling2d_26',
 'conv2d_115',
 'conv2d_116',
 'max_pooling2d_27',
 'conv2d_117',
 'conv2d_118',
 'max_pooling2d_28',
 'conv2d_119',
 'conv2d_120',
 'dropout_8',
 'max_pooling2d_29',
 'conv2d_121',
 'conv2d_122',
 'dropout_9',
 'up_sampling2d_14',
 'conv2d_123',
 'concatenate_14',
 'conv2d_124',
 'conv2d_125',
 'up_sampling2d_15',
 'conv2d_126',
 'concatenate_15',
 'conv2d_127',
 'conv2d_128',
 'up_sampling2d_16',
 'conv2d_129',
 'concatenate_16',
 'conv2d_130',
 'conv2d_131',
 'up_sampling2d_17',
 'conv2d_132',
 'concatenate_17',
 'conv2d_133',
 'conv2d_134',
 'conv2d_135',
 'conv2d_136']

In [84]:
from keras.layers import Input

inputs = Input(shape=(256, 256, 1))  # or whatever shape your images have


In [85]:
def conv_block(x, filters):
    x = Conv2D(filters, 3, activation='relu', padding='same')(x)
    x = Conv2D(filters, 3, activation='relu', padding='same')(x)
    return x


In [86]:
conv1 = conv_block(inputs, 64)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)


In [87]:
conv1, pool1 = encoder_block(inputs, 64)


In [88]:
up6 = Conv2D(512, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(drop5))
merge6 = concatenate([drop4, up6], axis=3)


In [89]:
conv5 = Conv2D(1, 1, activation='sigmoid')(conv5)


In [90]:
def encoder_block(x, filters):
    x = Conv2D(filters, 3, activation='relu', padding='same', kernel_initializer='he_normal')(x)
    x = Conv2D(filters, 3, activation='relu', padding='same', kernel_initializer='he_normal')(x)
    p = MaxPooling2D(pool_size=(2, 2))(x)
    return x, p


In [91]:
def decoder_block(x, skip, filters):
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(filters, 2, activation='relu', padding='same', kernel_initializer='he_normal')(x)
    x = concatenate([skip, x], axis=3)
    x = Conv2D(filters, 3, activation='relu', padding='same', kernel_initializer='he_normal')(x)
    x = Conv2D(filters, 3, activation='relu', padding='same', kernel_initializer='he_normal')(x)
    return x



In [92]:
output = Conv2D(1, 1, activation='sigmoid')(conv5)


In [93]:
model = build_unet()



In [94]:
import matplotlib.pyplot as plt

def visualize_prediction(model, X, Y, index=0):
    input_img = X[index]
    true_mask = Y[index]
    pred_mask = model.predict(np.expand_dims(input_img, axis=0))[0]

    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    axs[0].imshow(input_img.squeeze(), cmap='gray')
    axs[0].set_title("Input Image")
    axs[1].imshow(true_mask.squeeze(), cmap='gray')
    axs[1].set_title("Ground Truth")
    axs[2].imshow(pred_mask.squeeze(), cmap='gray')
    axs[2].set_title("Predicted Mask")
    for ax in axs:
        ax.axis('off')
    plt.tight_layout()
    plt.show()
